In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -U datasets
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U peft
!pip install -q -U trl
#!pip install -q -U git+https://github.com/huggingface/accelerate.git


In [ ]:
pip install -U bitsandbytes

In [ ]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                   # 4비트 로드 활성화
    bnb_4bit_quant_type="nf4",           # 양자화 방식 (예: "nf4" 또는 "fp4")
    bnb_4bit_use_double_quant=True,      # 이중 양자화 사용 여부
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 시 사용할 데이터 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,              # 리포지토리의 custom 코드를 실행하도록 설정
    quantization_config=quant_config,      # BitsAndBytesConfig 객체 전달
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)


In [ ]:
model

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Exaone 모델 구조에 맞게 target_modules 목록을 수정
lora_config = LoraConfig(
    r=4,                     # LoRA 가중치 행렬의 rank (값이 작을수록 trainable parameter 수가 줄어듦)
    lora_alpha=8,            # LoRA 스케일링 팩터
    lora_dropout=0.05,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'out_proj', 'c_fc_0', 'c_fc_1', 'c_proj'],
    bias='none',             # bias 파라미터 학습 여부: 'none', 'all', 'lora_only'
    task_type="CAUSAL_LM"     # 문자열로 task_type 지정 (이전에는 TaskType.CAUSAL_LM을 사용했음)
)

# 양자화된 모델 학습 전 전처리
model = prepare_model_for_kbit_training(model)

# PEFT를 사용해 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/train.csv", encoding = 'utf-8-sig')

In [ ]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

In [ ]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [ ]:
from datasets import Dataset

# combined_training_data가 Pandas DataFrame인 경우
dataset_hf = Dataset.from_pandas(combined_training_data.reset_index(drop=True))


In [ ]:
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 재발 방지 대책 및 향후 조치 계획만 간결하게 답변하세요.
- 서론, 배경 설명, 추가 설명 없이 핵심 내용만 전달하세요.
- 불필요한 형식(목차, 강조 표시, 리스트 등)을 사용하지 마세요.
- 한 문장 또는 간결한 문단으로 자연스럽게 작성하세요.
- 특수문자를 포함하지 마세요.

{context}

### 질문:
{question}

### 답변:
"""

In [ ]:
def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["question"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"text": prompt, "answer": data_point["answer"]}

# 기존 데이터셋의 모든 컬럼을 제거하고, generate_prompt에서 필요한 컬럼만 남깁니다.
remove_column_keys = list(dataset_hf.features.keys())
dataset_cvted = dataset_hf.shuffle(seed=42).map(generate_prompt, remove_columns=remove_column_keys)

In [ ]:
dataset_cvted

In [ ]:
print(dataset_cvted[0])

In [ ]:
# 토크나이징 완료된 데이터셋을 train/test로 분리
split_datasets = dataset_cvted.train_test_split(test_size=0.1, seed=42)

train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]  # 보통 검증 용도로 사용

In [ ]:
train_dataset[0]

In [ ]:
def tokenize_function(examples):
    outputs = tokenizer(examples["text"], truncation=True, max_length=512)
    return outputs


In [ ]:
remove_column_keys = train_dataset.features.keys()
train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

remove_column_keys = eval_dataset.features.keys()
test_dataset_tokenized = eval_dataset.map(tokenize_function, batched=True, remove_columns=remove_column_keys)

In [ ]:
train_dataset_tokenized

In [ ]:
test_dataset_tokenized

In [ ]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [ ]:
# 2. TrainingArguments에 evaluation_strategy와 eval_steps 추가
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=8,         # 각 디바이스당 배치 사이즈
    gradient_accumulation_steps=4,
    warmup_steps=1,
    max_steps=2000,
    learning_rate=2e-4,                     # 학습률
    bf16=True,                              # bf16 사용 (하드웨어 지원 확인 필요)
    output_dir="/content/drive/MyDrive/model_weight",
    optim="paged_adamw_8bit",               # 8비트 AdamW 옵티마이저
    logging_steps=50,                       # 로깅 빈도
    save_total_limit=3,                     # 저장할 체크포인트의 최대 수
    evaluation_strategy="steps",            # 일정 스텝마다 평가 실행
    eval_steps=50                           # 평가 주기 (50 스텝마다 평가)
)

# 3. SFTTrainer에 eval_dataset을 추가하여 수정
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=train_args,
    data_collator=collate_fn
)

In [ ]:
model.config.use_cache = False
trainer.train()

In [ ]:
# 저장 - > 이때 qLoRA를 이용해서 학습한 파라미터만 저장되서 작음
FINETUNED_MODEL = "qlora"
trainer.model.save_pretrained(FINETUNED_MODEL)

In [ ]:
from peft import PeftConfig

In [ ]:
#불러오기
FINETUNED_MODEL = "/content/qlora"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [ ]:
# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    peft_config.base_model_name_or_path
)

In [ ]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [ ]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

In [ ]:
# 샘플 TEST
prompt = dataset_cvted[2]['text']
# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)
outputs = pipe(
    prompt,
    do_sample=False,
    temperature=0.1,
    #top_k=0,
    #top_p=0.95,
    repetition_penalty=1.2,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
dataset_cvted[2]

In [ ]:
# test 데이터 전처리
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

# 1. CSV 파일 로드 (예: test.csv)
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")

# 데이터 전처리
test_df['공사종류(대분류)'] = test_df['공사종류'].str.split(' / ').str[0]
test_df['공사종류(중분류)'] = test_df['공사종류'].str.split(' / ').str[1]
test_df['공종(대분류)'] = test_df['공종'].str.split(' > ').str[0]
test_df['공종(중분류)'] = test_df['공종'].str.split(' > ').str[1]
test_df['사고객체(대분류)'] = test_df['사고객체'].str.split(' > ').str[0]
test_df['사고객체(중분류)'] = test_df['사고객체'].str.split(' > ').str[1]

# 2. train 데이터와 동일한 방식으로 prompt 생성
def generate_prompt_from_row(row):
    prompt = (
         f"작업 프로세스는 '{row['작업프로세스']}'이며 {row['인적사고']}발생, 사고 원인은 '{row['사고원인']}'입니다."
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
    )
    return {"prompt": prompt}

# DataFrame의 각 행에 대해 prompt 생성 (새로운 열로 추가)
test_df = test_df.apply(generate_prompt_from_row, axis=1)
test_df = pd.DataFrame(test_df.tolist())

# 3. HuggingFace Dataset으로 변환
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

def generate_prompt(data_point):
    # 추가 context가 필요한 경우 여기서 설정 (현재는 빈 문자열)
    context = ""
    question = data_point["prompt"]
    # prompt_template을 사용해 prompt를 생성합니다.
    prompt = prompt_template.format(context=context, question=question)
    # 모델 입력으로 사용할 프롬프트와 정답(평가용)만 반환합니다.
    return {"prompt": prompt,}

remove_column_keys = list(test_dataset.features.keys())
test_dataset = test_dataset.map(generate_prompt, remove_columns=remove_column_keys)

In [ ]:
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# 가정: test_dataset은 HuggingFace Dataset 객체이며, merged_model과 tokenizer가 이미 정의되어 있음.
# 예시: test_dataset.features = ['prompt']

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=64)

# 배치 사이즈 설정 (메모리 상황에 맞게 조정)
batch_size = 8
test_results = []

# HuggingFace Dataset은 slicing 시 dict 형태로 각 컬럼을 반환할 수 있음
for i in tqdm(range(0, len(test_dataset), batch_size)):
    # i부터 i+batch_size 까지 슬라이싱하여 배치 데이터 추출
    batch = test_dataset[i:i+batch_size]
    # "prompt" 컬럼의 리스트를 추출 (예: batch["prompt"]가 리스트 형태)
    batch_prompts = batch["prompt"]

    # 배치로 텍스트 생성 (한 번에 여러 프롬프트 처리)
    batch_outputs = pipe(
        batch_prompts,
        do_sample=False,
        temperature=0.1,
        repetition_penalty=1.2,
        add_special_tokens=True
    )

    # 각 프롬프트에 대해 첫 번째 생성 결과를 이용하여, 원본 프롬프트 이후의 텍스트만 추출
    for prompt, outputs in zip(batch_prompts, batch_outputs):
        generated_text = outputs[0]["generated_text"][len(prompt):]
        test_results.append(generated_text)

In [ ]:
# 생성된 텍스트 리스트에 대해 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

# sample_submission 파일 불러오기
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv', encoding='utf-8-sig')

# 최종 결과 저장: 두 번째 열에 생성 텍스트, 이후 열에 임베딩 벡터 할당
submission.iloc[:, 1] = test_results
submission.iloc[:, 2:] = pred_embeddings

print(submission.head())

# 최종 결과 CSV로 저장
submission.to_csv('exaone+QLoRA_V1.csv', index=False, encoding='utf-8-sig')
